![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Prompt Notebook with Chat - Prompt Lab Notebook v1.1.0
This notebook contains steps and code to demonstrate inferencing of prompts
generated in Prompt Lab in watsonx.ai with a chat format. It introduces Python API commands
for authentication using API key and prompt inferencing using WML API.

**Note:** Notebook code generated using Prompt Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Prompt Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.10.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Defining parameters of the Model object
* Using the Model object to generate response using the defined model id, parameters and the prompt input

# Setup

In [7]:
!pip install --upgrade 'chromadb==0.3.26' 'pydantic==1.10.0' sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.4/138.4 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 5.6 MB/s eta 0:00:00


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 80.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 977.6/977.6 kB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 53.6 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 83.2 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 78.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 73.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 88.4 MB/s eta 0:00:00:0

  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp310-cp310-linux_x86_64.whl size=187974 sha256=457ee6be7a158d391d467966f713c9fb0b244b1f8d0ed3bc1959c7f630f31ced
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/af/a9/3e/3e5d59ee41664eb31a4e6de67d1846f86d16d93c45f277c4e7
Successfully built hnswlib
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typing_extensions-4.4.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.11.0
    Uninstalling fsspec-2022.11.0:
      Successfully uninstalled fsspec-2022.11.0


## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [9]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://eu-de.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("OYkUFWdQzRYZgEpHEwJ05FLblo67CrftRFIzEbUvEGM8")
	}


# Inferencing
This cell demonstrated how we can use the model object as well as the created access token
to pair it with parameters and input string to obtain
the response from the the selected foundation model.

## Defining the model id
We need to specify model id that will be used for inferencing:


In [10]:
model_id = "sdaia/allam-1-13b-instruct"


## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [11]:
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 900,
    "repetition_penalty": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [12]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Defining the Model object
We need to define the Model object using the properties we defined so far:


In [13]:
from ibm_watsonx_ai.foundation_models import Model

model = Model(
	model_id = model_id,
	params = parameters,
	credentials = get_credentials(),
	project_id = project_id,
	space_id = space_id
	)


OYkUFWdQzRYZgEpHEwJ05FLblo67CrftRFIzEbUvEGM8········


## Defining the vector index
Initialize the vector index to query when chatting with the model.

In [14]:
from ibm_watsonx_ai.client import APIClient

wml_credentials = get_credentials()
client = APIClient(credentials=wml_credentials, project_id=project_id, space_id=space_id)

vector_index_id = "489f2f17-de49-4e59-9a06-51d4f72d5c94"
vector_index_details = client.data_assets.get_details(vector_index_id)
vector_index_properties = vector_index_details["entity"]["vector_index"]

from ibm_watsonx_ai.foundation_models.embeddings.sentence_transformer_embeddings import SentenceTransformerEmbeddings

emb = SentenceTransformerEmbeddings('sentence-transformers/all-MiniLM-L6-v2')

import subprocess
import gzip
import json
import chromadb
import random
import string

def hydrate_chromadb():
    data = client.data_assets.get_content(vector_index_id)
    content = gzip.decompress(data)
    stringified_vectors = str(content, "utf-8")
    vectors = json.loads(stringified_vectors)

    chroma_client = chromadb.Client()

    # make sure collection is empty if it already existed
    collection_name = "my_collection"
    try:
        collection = chroma_client.delete_collection(name=collection_name)
    except:
        print("Collection didn't exist - nothing to do.")
    collection = chroma_client.create_collection(name=collection_name)

    vector_embeddings = []
    vector_documents = []
    vector_metadatas = []
    vector_ids = []

    for vector in vectors:
        vector_embeddings.append(vector["embedding"])
        vector_documents.append(vector["content"])
        metadata = vector["metadata"]
        lines = metadata["loc"]["lines"]
        clean_metadata = {}
        clean_metadata["asset_id"] = metadata["asset_id"]
        clean_metadata["asset_name"] = metadata["asset_name"]
        clean_metadata["url"] = metadata["url"]
        clean_metadata["from"] = lines["from"]
        clean_metadata["to"] = lines["to"]
        vector_metadatas.append(clean_metadata)
        asset_id = vector["metadata"]["asset_id"]
        random_string = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))
        id = "{}:{}-{}-{}".format(asset_id, lines["from"], lines["to"], random_string)
        vector_ids.append(id)

    collection.add(
        embeddings=vector_embeddings,
        documents=vector_documents,
        metadatas=vector_metadatas,
        ids=vector_ids
    )
    return collection

chroma_collection = hydrate_chromadb()

def proximity_search( question ):
    query_vectors = emb.embed_query(question)
    query_result = chroma_collection.query(
        query_embeddings=query_vectors,
        n_results=vector_index_properties["settings"]["top_k"],
        include=["documents", "metadatas", "distances"]
    )

    documents = list(reversed(query_result["documents"][0]))

    return "\n".join(documents)


OYkUFWdQzRYZgEpHEwJ05FLblo67CrftRFIzEbUvEGM8········


/opt/conda/envs/Python-RT23.1/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/opt/conda/envs/Python-RT23.1/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedba

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Collection didn't exist - nothing to do.


## Defining the inferencing input for chat
Foundation models supporting chat accept a system prompt that instructs the model on how to conduct the dialog. They also accept previous questions and answers to give additional context when inferencing. Each model has it's own string format for constructing the input.

Let us provide the input we got from the Prompt Lab and format it for the selected model:


In [27]:
import io
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='sIU5ReLdi_OLTTmTbsf87i92TasxpeEYYSTUUwuixj4y',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.eu-de.cloud-object-storage.appdomain.cloud')

bucket = 'afuq-donotdelete-pr-3trjp41xr12htq'
object_key = 'response.txt'

# load data of type "text/plain" into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']
expected_responses = streaming_body_1.read().decode("utf-8").splitlines()
#df = pd.read_table(io.BytesIO(streaming_body_1.read()))

In [28]:
expected_responses

['الخوارزمي واختراع الجبر,في يوم من الأيام، كان هناك عالم يدعى الخوارزمي. عاش الخوارزمي في بغداد القديمة، وبدأ في التفكير في حل مسائل رياضية معقدة. اخترع علماً جديداً يسمى الجبر، الذي نستخدمه اليوم لحل المعادلات. في تلك الأوقات، لم يكن هناك حواسيب أو آلات حساب، ولكن الخوارزمي استخدم عقله وذكاءه ليبني أسساً تساعد العلماء والمبتكرين في المستقبل.',
 'البيروني وحساب محيط الأرض,في يوم من الأيام، جلس العالم الكبير البيروني ينظر إلى السماء ويتأمل الأرض. بدأ يفكر، كيف يمكنه حساب محيط الأرض باستخدام أدوات بسيطة. قام بقياس زوايا الشمس واستخدم علم الرياضيات ليحسب المسافة بين المدن. وكانت النتيجة مذهلة، فقد كانت حساباته قريبة جداً من القياسات الحديثة التي نعرفها اليوم!',
 'ابن سينا ومعادلات الأعداد,في أحد الأيام، كان العالم الكبير ابن سينا يجلس في مكتبه يتأمل كتب الرياضيات القديمة. بدأ يفكر في الأعداد وكيفية استخدامها لحل مشكلات الحياة اليومية. كتب معادلات رياضية كانت تساعد الأطباء على حساب الجرعات الطبية بشكل دقيق، وكان من أوائل العلماء الذين استخدموا الرياضيات في الطب.',
 'ابن الهيثم ونظرية الضو

In [41]:
from sentence_transformers import SentenceTransformer, util

# إعداد النموذج التضميني
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# وظيفة لحساب التشابه الدلالي بين الإجابات المتوقعة والفعلية
def calculate_semantic_accuracy(expected_responses, actual_responses):
    expected_embeddings = embedding_model.encode(expected_responses, convert_to_tensor=True)
    actual_embeddings = embedding_model.encode(actual_responses, convert_to_tensor=True)

    similarities = util.pytorch_cos_sim(expected_embeddings, actual_embeddings)
    max_similarity = similarities.max().item()
    
    return max_similarity * 100

In [42]:
prompt_input = """
__grounding__

<<SYS>>
اذا قلت مرحبا او اي ترحيب ابيك ترد بسؤالي عن نوع القصه
ثم اكتب قصة تفاعلية للأطفال تعتمد على اختيار الطفل 
القواعد :
1- اذا ادخل المستخدم كلمة ترحيبية مثل مرحبا قم الترحيب به بشكل ودي  ثم اسئله عن نوع القصة التي يرغب في انشائها ثم بعد ذلك  انشاء الجزء الاول من القصة ثم  قم بسؤاله عن القصة ثم خذ الرد واكمل مجريات القصة بناء على رد المستخدم 
2-  استمر في سرد القصة بهذه الطريقة حتى الوصول إلى نهاية مرضية للقصة مع  إضافة ملخص وحكمة بعد الانتهاء من القصة بالكامل.
3- لاتجعل القصه طويله للغايه

 ابدأ بمقدمة مشوقة تحتوي على احداث القصة ومغامراتها والشخصيات الأساسيه.
 ثم ضع سؤالاً  للأطفال حول أحداث القصة أو القرارات التي يمكن للشخصيات اتخاذها.
 بعد ذلك انتظر إجابة الطفل على السؤال ثم استخدم الإجابة لإكمال القصة بناءً على اختياراتهم.

تنبيه:
يجب ان تنتظر اجابة المستخدم.
يجب أن تكون القصة مكتوبة بلغة بسيطة وواضحة تناسب الأطفال.
بعد كل جزء من القصة ضع سؤالًا جديدًا يتعلق بالأحداث وانتظر إجابة المستخدم.
 استمر في سرد القصة بهذه الطريقة حتى الوصول إلى نهاية مرضية للقصة مع  إضافة ملخص وحكمة بعد الانتهاء من القصة بالكامل.

المدخل:
اياً كان ماسيكتبه المستخدم لابد أن تطلب منه نوع القصة.

الإخراج المتوقع:
تقوم بسرد قصة بنوع [نوع القصة]. تبدأ بمقدمة لطيفة و مشوقة وتنتهي بنهاية سعيدة مع ملخص وحكمة تناسب الأطفال.
القصة يجب أن تحتوي على عدة أجزاء متتالية مع أسئلة بعد كل جزء، وتُبنى الأحداث على القرارات التي يتخذها المستخدم.

مثال عملي:

نوع القصة:  تاريخي
الجزء الأول من القصة: كان المسلمون يعيشون في المدينة المنورة، وكانوا في أمان. لكن عندما علموا بخطة الأعداء لتهديدهم، قرر النبي ﷺ أن يحمي المدينة. فقال الصحابي سلمان الفارسي: 'يا رسول الله، في بلاد فارس كنا نحفر خندقًا لتحمي مدينتنا من الأعداء.' فوافق النبي ﷺ على هذه الفكرة.
ماذا تعتقد أن المسلمين يجب أن يفعلوا الآن؟ هل يستمرون في حفر الخندق، أم يجب عليهم التوقف وأخذ استراحة؟
القواعد:
بناءً على إجابة الطفل، يتم تحديد ما يحدث بعد ذلك في القصة بعد ذلك، ضع سؤالًا جديدًا بناءً على الجزء الذي تم اختياره، وكرر هذه العملية.
.قم بإستخدام البيانات ايضاً لانشاء القصة

تنبيه:
بعد كل جزء، انتظر إجابة الطفل على السؤال، ثم أكمل القصة بناءً على ما يختاره. تأكد من أن القصة لا تستمر إلا بعد تلقي إجابة.
<</SYS>>

 مرحبا [/INST]  مرحبًا! سأكون سعيدًا لمساعدتك في إنشاء قصة تفاعلية للأطفال. ما نوع القصة التي ترغب في إنشائها اليوم؟ هل تفضل قصة تاريخية، مغامرة، خيالية، أم تعليمية؟  </s><s> [INST] تاريخية [/INST]  رائع! سأقوم بإنشاء قصة تاريخية تفاعلية لك. لنبدأ بمقدمة مشوقة:

في زمن بعيد، في مدينة بغداد العظيمة، عاش عالم عظيم يدعى ابن سينا. كان ابن سينا طبيبًا وفيلسوفًا وكيميائيًا بارعًا. كتب كتابًا مشهورًا يُدعى "القانون في الطب"، الذي ظل يُدرس في الجامعات لسنوات عديدة. بفضل ابن سينا، تعلم الأطباء كيفية علاج المرضى بشكل أفضل.

الآن، دعونا نبدأ القصة. في يوم من الأيام، كان ابن سينا يسير في شوارع بغداد عندما سمع عن مرض غريب ينتشر بين الناس. قرر أن يبحث عن علاج لهذا المرض ويجد طريقة لمساعدة الناس.

ماذا يجب أن يفعل ابن سينا أولاً؟ هل يجب أن يبحث عن معلومات حول المرض، أم يجرب تحضير دواء جديد، أم يستشير أطباء آخرين؟

يرجى اختيار الخيار الذي تعتقد أنه الأفضل لابن سينا، وسأكمل القصة بناءً على اختيارك.  </s><s> [INST] معلومات حول المرض [/INST]  قرر ابن سينا أن يبدأ بجمع معلومات حول المرض الغريب الذي ينتشر بين الناس. توجه إلى مكتبة بغداد الكبيرة وبدأ في قراءة الكتب والمخطوطات الطبية. قضى ساعات طويلة في البحث والدراسة، محاولا فهم طبيعة المرض وأسبابه.

بينما كان ابن سينا يبحث في المكتبة، لاحظ وجود كتاب قديم ومتهالك. كان عنوان الكتاب "الشفاء"، وهو كتاب كتبه الفيلسوف اليوناني الشهير جالينوس. كان ابن سينا يعلم أن جالينوس كان طبيبًا بارعًا، لذا قرر أن يستعير الكتاب ويبدأ في قراءته.

بينما كان يقرأ الكتاب، وجد ابن سينا معلومات قيمة عن مرض مشابه للمرض الذي يحاول علاجه. أدرك أن جالينوس قد وصف أعراض المرض وطرق علاجه. شعر ابن سينا بالامتنان لجالينوس على المعرفة التي قدمها وقرر أن يستفيد من هذه المعلومات في بحثه عن علاج للمرض.

ماذا يجب أن يفعل ابن سينا بعد ذلك؟ هل يجب أن يعود إلى المكتبة للبحث عن مزيد من المعلومات، أم يبدأ في تحضير دواء جديد بناءً على ما تعلمه، أم يستشير أطباء آخرين؟

يرجى اختيار الخيار الذي تعتقد أنه الأفضل لابن سينا، وسأكمل القصة بناءً على اختيارك.  </s><s> [INST]"""


## Execution
Let us now use the defined Model object, pair it with the input, and generate the response to your question:


In [45]:
question = input("Question: ")
grounding = proximity_search(question)
formattedQuestion = f"""<s> [INST] {question} [/INST]"""
prompt = f"""{prompt_input}{formattedQuestion}"""
generated_response = model.generate_text(prompt=prompt.replace("__grounding__", grounding), guardrails=False)
print(f"AI: {generated_response}")
actual_responses = [generated_response]

Question: اريد قصه تاريخية
AI:  بالطبع! سأقوم بإنشاء قصة تاريخية لك. لنبدأ بمقدمة مشوقة:

في زمن قديم، في مدينة الإسكندرية العظيمة، عاش عالم مشهور يدعى إقليدس. كان إقليدس رياضيًا ومهندسًا بارعًا، واشتهر بكتابه "العناصر" الذي يعتبر أحد أهم الكتب في تاريخ الرياضيات.

الآن، دعونا نبدأ القصة. في يوم من الأيام، كان إقليدس يسير في شوارع الإسكندرية عندما سمع عن مشكلة هندسية معقدة تواجه المهندسين المحليين. قرر إقليدس أن يساعد في حل هذه المشكلة ويجد طريقة لمساعدة المهندسين.

ماذا يجب أن يفعل إقليدس أولاً؟ هل يجب أن يبحث عن معلومات حول المشكلة، أم يجرب تطبيق مبادئ الهندسة التي تعلمها، أم يستعين بعلماء آخرين للمساعدة؟

يرجى اختيار الخيار الذي تعتقد أنه الأفضل لإقليدس، وسأكمل القصة بناءً على اختيارك. 


In [46]:
semantic_accuracy = calculate_semantic_accuracy(expected_responses, actual_responses)
print(f"الدقة الدلالية: {semantic_accuracy:.2f}%")

الدقة الدلالية: 95.16%


# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2023 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  